<a href="https://colab.research.google.com/github/sureshrgmi/LandTech--Suresh-Regmi/blob/main/Q3%20-%20Sourcing%20Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The following code shows how to gather data from a specific API (PlanHat in this case), Load it into pandas dataframe, perform transformation if necessary and finally load them to a Redshift data warehouse. The process follows these simple stps and contains placeholders instead of actual credentials which could be changed based on the environment you are working on.


Steps

1. Import necessary libraries such as requests, pandas, psycopg2
2. Define variables for the api url, api token (if necessary), endpoint you want to work with and database credentials.
3. Get the data from the API and load it into pandas dataframe
4. Perform exploartory analysis and data transformation on the reponse data in the dataframe (I prefer performing this task at the later stage using SQL once the data is loaded into the database/warehouse)
5. Load the data into the data warehouse utilizing the credentials

In [ ]:
# Import the necessary libraries
import requests
import pandas as pd
import psycopg2

# PlanHat API credentials and endpoints
planhat_api_url = 'https://api.planhat.com/v1/'
planhat_api_token = 'YOUR_PLANHAT_API_TOKEN'  # Replace with your PlanHat API token

# Redshift database credentials
redshift_host = 'YOUR_REDSHIFT_HOST'  # Replace this with the hostname/url
redshift_port = '5439'  # Using default port here
redshift_db = 'YOUR_REDSHIFT_DB'  # Replace this with the database name
redshift_user = 'YOUR_REDSHIFT_USER'  # Redshift username
redshift_password = 'YOUR_REDSHIFT_PASSWORD'  # Redshift password

# Define the API endpoint for the PlanHat data you want to get
planhat_endpoint = 'YOUR_PLANHAT_ENDPOINT'  # Replace this with the specific endpoint you want to get

# Make an API request to PlanHat
headers = {
    'Authorization': f'Bearer {planhat_api_token}'
}

response = requests.get(planhat_api_url + planhat_endpoint, headers=headers)

# Check if the API request was successful
if response.status_code == 200:
    # Convert the JSON response to a Pandas DataFrame
    data = response.json()
    df = pd.DataFrame(data)

    # We can do the data transformation using libraries such as Pandas and NumPy
    # Personally, I prefer loading this data into database table as it is and
    # performing the data transformation there

    # Connect to Redshift
    conn = psycopg2.connect(
        host=redshift_host,
        port=redshift_port,
        dbname=redshift_db,
        user=redshift_user,
        password=redshift_password
    )

    # Create a cursor
    cur = conn.cursor()

    # Define the Redshift table where you want to insert the data
    redshift_table_name = 'YOUR_REDSHIFT_TABLE'  # Replace with the actual table name

    # Inserting data into the Redshift table
    df.to_sql(name=redshift_table_name, con=conn, if_exists='replace', index=False)

    # Commit the transaction and close the cursor and connection
    conn.commit()
    cur.close()
    conn.close()

    print(f'Data successfully loaded into Redshift table: {redshift_table_name}')
else:
    print(f'Error: Unable to fetch data from PlanHat API. Status code: {response.status_code}')
